In [1]:
!ls

assignment_2.ipynb  brown.txt  question.pdf


In [2]:
import string, re
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm.auto import tqdm
import numpy as np
from collections import Counter
import random

In [3]:
class txt_data():
    '''
    Master class to load dataset
    '''
    def __init__(self,path):
        self.path = path
        
        # list of sentences of list of tuples defining pos_tag and words.
        self.data = self.___sent_to_tags___(self.___open___(self)) 
        random.shuffle(self.data)
        self.train =  self.data[:int(len(self.data)*0.8)]
        self.val = self.data[int(len(self.data)*0.8):]
        
        self.pos_tags = [i for i in set([j[0] for i in self.data for j in i])] #list of unique pos_tags
        self.vocab = [i for i in set([j[1] for i in self.data for j in i])] # list of vocab
            
    
    def ___open___(self,path):
        '''
        To read a text file
        path: path to the text file
        '''
        with open(self.path, "r") as f:
            return f.readlines()
    
    def ___sent_to_tags___(self,sents):
        '''
        '''
        dummy = []
        for i in tqdm(sents):
            d = []
            for j in i.split():
                try: d.append((j.split('_')[1],j.split('_')[0].lower()))
                except: pass #print(f"--{i}--{j}\n")
            dummy.append(d)
        return dummy

In [4]:
path = "brown.txt"
brown = txt_data(path)

In [5]:
len(brown.data), len(brown.train), len(brown.val), len(brown.train)+len(brown.val)

(55145, 44116, 11029, 55145)

In [151]:
brown.val

[[('AT', 'the'),
  ('OD', 'third'),
  ('NN', 'time'),
  ('BEDZ', 'was'),
  ('IN', 'on'),
  ('AT', 'the'),
  ('NN', 'floor'),
  ('IN', 'of'),
  ('AT', 'the'),
  ('NP', 'beverly'),
  ('NP', 'hilton'),
  ('NN', 'ballroom'),
  ('CC', 'and'),
  ('IN', 'for'),
  ('AT', 'the'),
  ('JJ', 'critical'),
  ('NNS', 'eyes'),
  ('CC', 'and'),
  ('NNS', 'tongues'),
  ('IN', 'of'),
  ('NNS', 'judges'),
  ('.', '.')],
 [('AT', 'the'),
  ('CD', 'two'),
  ('IN', 'of'),
  ('PPO', 'them'),
  ('IN', 'against'),
  ('AT', 'the'),
  ('NN', 'world'),
  ('.', '.')],
 [('(', '('),
  ('CC', 'but'),
  ('DT', 'that'),
  ('BEZ', 'is'),
  ('DT', 'another'),
  ('NN', 'story'),
  ('.', '.')],
 [('IN', 'in'),
  ('NN', 'addition'),
  ('IN', 'to'),
  ('AT', 'the'),
  ('NN', 'incompleteness'),
  ('IN', 'of'),
  ('NN', 'science'),
  ('CC', 'and'),
  ('AT', 'the'),
  ('NN', 'completeness'),
  ('IN', 'of'),
  ('NN', 'metaphysics'),
  (',', ','),
  ('PPSS', 'they'),
  ('VB', 'differ'),
  ('IN', 'in'),
  ('CS', 'that'),
  ('NN', 

In [6]:
def prob(dict_): return {k:np.array(v)/sum(v) for k,v in dict_.items()}

In [7]:
def e_matrix(data,vocab):
    dummy = {}
    data = [i for j in data for i in j] # list of pos_tag and word pair
    print(data[:4])
    for i in data:
        if i[0] in dummy.keys():
            dummy[i[0]].append(i[1])
        else: dummy[i[0]] = [i[1]]
    e_matrix = {}
    for t,v in tqdm(dummy.items()): 
        d = []
        count = Counter(v)
        for j in vocab:
            d.append(count[j])
        e_matrix[t] = d
    return e_matrix
e_prob = prob(e_matrix(brown.data, brown.vocab))

[('IN', 'in'), ('AT', 'the'), ('AP', 'latter'), ('NN', 'research')]


In [8]:
def t_matrix(data, pos_tags, n_gram=2):
    dummy = {}
    data = [[j[0]  for j in i] for i in data] # list of tag 
    print(data[:2],'\n')
    start = Counter([j for i in data for j in i])
    tags = [i[j:j+n_gram] for i in data for j in range(len(i) - n_gram + 1)] #list of n_grams of tags
#     print(tags[:10])
    tags_ = {i:[] for i in pos_tags}
    for i in tags:
        tags_[i[0]].append(i[1])
    t_matrix = {}
    t_matrix["start"] = [start[i] for i in pos_tags]
    for tp,tf in tqdm(tags_.items()): 
        d = []
        count = Counter(tf)
        for j in pos_tags:
            d.append(count[j])
        t_matrix[tp] = d
    return t_matrix
t_prob = prob(t_matrix(brown.data, brown.pos_tags, n_gram=2))

[['IN', 'AT', 'AP', 'NN', 'NN', ',', 'NN', 'BEZ', 'JJ', 'IN', 'CD', 'NP', 'NNS', 'VBN', 'IN', 'CD', 'CC', 'IN', 'CD', 'NNS', 'VBN', 'IN', 'CD', '.'], ['WQL', 'JJ', 'PPS', 'BEDZ', 'IN', 'JJ', 'NNS', '.']] 



# Confusion matrix

In [93]:
expected = [1, 1, 0,2,2,2,3,3,4,0,0,4,4,4,4,4,4,4,4]
predicted = [1, 0, 0,2,1,2,3,3,3,4,3,3,4,3,4,4,4,4,4]

# expected = [1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
# predicted = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0]

Counter(expected)

Counter({1: 2, 0: 3, 2: 3, 3: 2, 4: 9})

In [105]:
dummy = {str(i):{str(j) :0 for j in set(expected)} for i in set(expected) }
for r, i in enumerate(expected):
    dummy[str(i)][str(predicted[r])] +=1

conf_m = np.array([[j for j in i.values()] for i in dummy.values()]) # confusion matrix
dummy, conf_m

({'0': {'0': 1, '1': 0, '2': 0, '3': 1, '4': 1},
  '1': {'0': 1, '1': 1, '2': 0, '3': 0, '4': 0},
  '2': {'0': 0, '1': 1, '2': 2, '3': 0, '4': 0},
  '3': {'0': 0, '1': 0, '2': 0, '3': 2, '4': 0},
  '4': {'0': 0, '1': 0, '2': 0, '3': 3, '4': 6}},
 array([[1, 0, 0, 1, 1],
        [1, 1, 0, 0, 0],
        [0, 1, 2, 0, 0],
        [0, 0, 0, 2, 0],
        [0, 0, 0, 3, 6]]))

In [147]:
pre = np.sum(conf_m,axis=0)
rec = np.sum(conf_m,axis=1)
#classwise 
precision = [i[r] for r,i in enumerate(conf_m)]/pre 
recall = [i[r] for r,i in enumerate(conf_m)]/rec 
print("--",precision, recall)

#overall macro
print("macro--",sum(precision)/ len(precision), sum(recall)/len(recall))

#overall micro
precision = sum([i[r] for r,i in enumerate(conf_m)])/np.sum(pre) 
recall = sum([i[r] for r,i in enumerate(conf_m)])/np.sum(rec)
print("micro--",precision, recall)

-- [0.5        0.5        1.         0.33333333 0.85714286] [0.33333333 0.5        0.66666667 1.         0.66666667]
macro-- 0.6380952380952382 0.6333333333333333
micro-- 0.631578947368421 0.631578947368421


In [148]:
from sklearn.metrics import confusion_matrix
results = confusion_matrix(expected, predicted)
from sklearn.metrics import precision_score, recall_score
precision_score(expected, predicted, average =None), recall_score(expected, predicted, average =None)

(array([0.5       , 0.5       , 1.        , 0.33333333, 0.85714286]),
 array([0.33333333, 0.5       , 0.66666667, 1.        , 0.66666667]))

In [149]:
precision_score(expected, predicted, average ="macro"), recall_score(expected, predicted, average ="macro")

(0.6380952380952382, 0.6333333333333333)

In [150]:
precision_score(expected, predicted, average ="micro"), recall_score(expected, predicted, average ="micro")

(0.631578947368421, 0.631578947368421)